<a href="https://colab.research.google.com/github/armandossrecife/my_validation3/blob/main/my_analysis_hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup do Ambiente de Análise

In [1]:
!rm -rf *.log
!rm -rf *.txt
!rm -rf *.xlsx
!rm -rf my_issues
!rm -rf hadoop

In [2]:
print('Install Pydriller.')
!pip install pydriller > install_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('Install SQLite in Linux')
!sudo apt install -y sqlite3 > install_sqlite.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install Pydriller.
Install gitpython.
Install Jira Python lib.
Install SQLite in Linux


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
All depenpencies installed!
Details in install.log


## Importa bibliotecas necessárias

In [3]:
import os
from datetime import datetime, timedelta
import tqdm
from pydriller import Repository
from jira import JIRA
import pandas as pd
import re
import scipy.stats as stats
import random
import shutil
import time

## Variáveis Globais

In [4]:
my_repository = 'hadoop'
url_to_repository = 'https://github.com/apache/hadoop.git'
os.environ['MY_REPOSITORY'] = url_to_repository

JIRA_SERVER = 'https://issues.apache.org/jira'
ISSUE_TRACKER_PROJECT = 'HADOOP'

# Credentials
os.environ['USERNAME'] = ''
os.environ['PASSWORD'] = ''
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

lista_arquivos_criticos = ['Configuration.java', 'Writable.java', 'StringUtils.java', 'FSDataOutputStream.java', 'BytesWritable.java', 'WritableComparable.java', 'DatanodeProtocol.java', 'ClientProtocol.java', 'FSNamesystem.java', 'DataNode.java','BlockManager.java', 'ResourceScheduler.java', 'ContainerManager.java', 'FairScheduler.java','CapacityScheduler.java', 'NodeManager.java', 'Job.java', 'Mapper.java', 'Reducer.java', 'InputFormat.java', 'OutputFormat.java']

## Clona o Repositório

In [5]:
print(f'Clona o repositório : {url_to_repository}')
!git clone $MY_REPOSITORY

Clona o repositório : https://github.com/apache/hadoop.git
Cloning into 'hadoop'...
remote: Enumerating objects: 4178088, done.
remote: Counting objects: 100% (10274/10274), done.
remote: Compressing objects: 100% (2642/2642), done.
remote: Total 4178088 (delta 8658), reused 8971 (delta 7488), pack-reused 4167814
Receiving objects: 100% (4178088/4178088), 2.12 GiB | 20.59 MiB/s, done.
Resolving deltas: 100% (3011322/3011322), done.
Updating files: 100% (15308/15308), done.


In [13]:
!cd hadoop && git log --pretty="%H %s" > all_commits_msg.txt
!cd hadoop && git log --pretty="%H;%ai;%s" > all_commits_full.txt

# Dados do 1o commit e do ultimo commit
!echo "Último commit: "
!head /content/hadoop/all_commits_full.txt -n 1
!echo "Primeiro commit: "
!tail /content/hadoop/all_commits_full.txt -n 1

Último commit: 
a32097a921b6a256c82ee8c2a83aa1990c635e0d;2023-11-09 10:14:14 +0100;HADOOP-18954. Filter NaN values from JMX json interface. (#6229).
Primeiro commit: 
5128a9a453d64bfe1ed978cf9ffed27985eeef36;2009-05-19 04:20:40 +0000;HADOOP-4687 Moving src directories on branch


## Funções de apoio

In [7]:
def get_all_commits_by_range(initial_date, final_date, repository_name):
  """Extracts information from a date range of commits
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commit's date, commit's line, commit's file, a list of files modified in the commit, a list of all diffs from modified files
  """
  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
      list_of_modified_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_m_diff = {}
        if m is not None:
          list_of_modified_files.append(m.filename)
          dict_m_diff[m.filename] = m.diff
          list_dict_of_diff_modified_files.append(dict_m_diff)
      data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
      element = commit.msg, data_commit, commit.lines, commit.files, list_of_modified_files, list_dict_of_diff_modified_files
      dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

def get_commits_by_range_and_critical_files(initial_date, final_date, critical_files, repository_name):
  """Extracts information from a date range of commits, focusing on critical files.
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
      critical_files: A list of critical file paths. ex: ['StorageService.java', 'ColumnFamilyStore.java']
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commits's date, commit's line, commit's file, a list of critical files modified in the commit, a list_of_modified_files, a dictionary of diff [filename]:filename.diff, a list of all diffs from modified files
        commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
  """

  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress critical files commit anlysis"):
      list_of_critical_files_modified = []
      list_of_modified_files = []
      list_dict_of_diff_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_diff_file = {}
        list_of_modified_files.append(m.filename)
        list_dict_of_diff_modified_files.append(m.diff)
        if m.filename in critical_files:
          list_of_critical_files_modified.append(m.filename)
          dict_diff_file[m.filename] = m.diff
          list_dict_of_diff_files.append(dict_diff_file)
      if len(list_of_critical_files_modified) > 0:
        data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
        element = commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
        dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

In [8]:
class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments, created_date=None, updated_date=None, resolved_date=None):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments
    self.created_date = created_date
    self.updated_date = updated_date
    self.resolved_date = resolved_date

  def get_comments(self):
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

def analyze_jira_all_issues(project, all_issues):
    """Extracts all issues from Issue Tracker
        Args:
          project: Issue Tracker Project name, example: CASSANDRA
          all_issues: a block of issues
        Returns:
          A list of JiraIssues
    """
    # Create an instance of JiraIssues to manage all issues
    my_all_issues = JiraIssues(project, [])
    total_items = len(all_issues)

    # Iterate through the fetched issues
    for issue in tqdm.tqdm(all_issues, total=total_items, desc='Progress jira all issues analysis'):
        issue_key = issue.key
        issue_summary = issue.fields.summary
        issue_description = issue.fields.description
        issue_comments = [comment.body for comment in issue.fields.comment.comments]
        issue_created_date = issue.fields.created
        issue_updated_date = None
        issue_resolved_date = issue.fields.resolutiondate

        # Check for SATD keywords in the issue's summary, description, and comments
        if issue_summary is None:
            issue_summary = ""
        if issue_description is None:
            issue_description = ""
        if issue_comments is None:
            issue_comments = ""
        if issue_created_date == None:
            issue_created_date = ""
        if issue_updated_date == None:
            issue_updated_date = ""
        if issue_resolved_date == None:
            issue_resolved_date = ""

        issue_type = issue.fields.issuetype.name
        issue_status = issue.fields.status.name
        issue_priority = issue.fields.priority.name

        # Create a JiraIssue instance for the SATD issue
        my_issue = JiraIssue(issue_key, issue_summary, issue_type, issue_status, issue_priority, issue_description, issue_comments, issue_created_date, issue_updated_date, issue_resolved_date)

        my_all_issues.add_issue(my_issue)

    return my_all_issues

def convert_date_jira_to_datetime(jira_date):
  regex = r"(\d{4})-(\d{2})-(\d{2})"
  match = re.match(regex, jira_date)

  if match:
      year, month, day = match.groups()
      datetime_object = datetime(year=int(year), month=int(month), day=int(day))
      return datetime_object
  else:
      return None

def convert_issues_to_dataframe(all_real_issues):
  l_issue_key_aux, l_issue_type_aux, l_issue_summary_aux, l_issue_description_aux, l_issue_status_aux, l_issue_priority_aux, l_issue_comments_aux = [], [], [], [], [], [], []
  l_issue_created_date, l_issue_resolved_date = [], []

  for issue in all_real_issues.get_issues():
    l_issue_key_aux.append(issue.key)
    l_issue_type_aux.append(issue.issue_type)
    l_issue_summary_aux.append(issue.summary)
    l_issue_description_aux.append(issue.description)
    l_issue_status_aux.append(issue.status)
    l_issue_priority_aux.append(issue.priority)
    texto_aux = ""
    for item in issue.get_comments():
      texto_aux = texto_aux + str(item) + "\n"
    l_issue_comments_aux.append(texto_aux)
    created_date_temp = convert_date_jira_to_datetime(issue.created_date)
    l_issue_created_date.append(created_date_temp)
    resolved_date_temp = convert_date_jira_to_datetime(issue.resolved_date)
    l_issue_resolved_date.append(resolved_date_temp)

  dict_all_reall_issues_in_commits_detailed = {
  'issue_key': l_issue_key_aux,
  'issue_type':l_issue_type_aux,
  'status':l_issue_status_aux,
  'priority':l_issue_priority_aux,
  'summary':l_issue_summary_aux,
  'description':l_issue_description_aux,
  'comments':l_issue_comments_aux,
  'created_date': l_issue_created_date,
  'resolved_date': l_issue_resolved_date
  }

  df_all_reall_issues_in_commits_detailed = pd.DataFrame(dict_all_reall_issues_in_commits_detailed)
  return df_all_reall_issues_in_commits_detailed

In [9]:
def convert_commits_to_dataframe(dict_of_commits):
  '''
  v[0] = commit.msg,
  v[1] = data_commit,
  v[2] = commit.lines,
  v[3] = commit.files,
  v[4] = list_of_critical_files_modified,
  v[5] = list_of_modified_files,
  v[6] = list_dict_of_diff_files,
  v[7] = list_dict_of_diff_modified_files
  '''
  l_commit_hash, l_commit_msg, l_commit_data, l_commit_lines, l_commit_files, l_commit_critical_files, l_commit_modified_fies, l_commit_diff_files, l_commit_diff_modified_files = [], [], [], [], [], [], [], [], []
  for k, v in dict_of_commits.items():
    l_commit_hash.append(k)
    l_commit_msg.append(v[0])
    l_commit_data.append(v[1])
    l_commit_lines.append(v[2])
    l_commit_files.append(v[3])
    l_commit_critical_files.append(v[4])
    l_commit_modified_fies.append(v[5])
    l_commit_diff_files.append(v[6])
    l_commit_diff_modified_files.append(v[7])

  dict_of_commits_aux = {
      'hash': l_commit_hash,
      'msg': l_commit_msg,
      'date': l_commit_data,
      'lines': l_commit_lines,
      'files': l_commit_files,
      'critical_files': l_commit_critical_files,
      'modified_files': l_commit_modified_fies,
      'diff_files': l_commit_diff_files,
      'diff_files_modified_files': l_commit_diff_modified_files
  }

  df_commits = pd.DataFrame(dict_of_commits_aux)
  return df_commits

In [10]:
def find_issues_id_by_project(input_string: str, project: str) -> list[str]:
    """Finds all Cassandra issue ID patterns in the input string.
    Args:
        input_string: The input string.
        project: The pattern related to project name, for example: CASSANDRA project name
    Returns:
        A list of project issue IDs, if found; otherwise, an empty list.
    """
    # Try to find all Cassandra issue ID patterns in the input string
    matches = re.findall(r"({0}-\d+)".format(project), input_string)
    # Return an empty list if no matches are found
    if not matches:
        return []
    # Convert the list of matches to a set to remove duplicates
    set_matches = set(matches)
    # Convert the set of matches back to a list
    list_unique_matches = list(set_matches)

    # Return the list of matched Cassandra issue IDs
    return list_unique_matches

def get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files):
  dict_issues_in_commits = {}
  for index in df_commits_with_critical_files.index:
    l_issues_in_commit = find_issues_id_by_project(input_string=df_commits_with_critical_files.msg[index], project=ISSUE_TRACKER_PROJECT)
    if len(l_issues_in_commit) > 0:
      commit_hash = df_commits_with_critical_files.hash[index]
      dict_issues_in_commits[commit_hash] = l_issues_in_commit

  list_issue_commits, list_issue_issues = [], []
  for k, v in dict_issues_in_commits.items():
    list_issue_commits.append(k)
    for issue in v:
      if issue not in list_issue_issues:
        list_issue_issues.append(issue)

  df_aux  = df_all_reall_issues_in_commits_detailed.copy()
  df_issues_in_commits_with_critical_classes = df_aux[df_aux['issue_key'].isin(list_issue_issues)]

  return dict_issues_in_commits, df_issues_in_commits_with_critical_classes

def calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size):
    # Calculate the Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # Calculate the sample size formula
    sample_size = ((z_score**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # Adjust for finite population
    if population_size:
        sample_size = sample_size / (1 + ((sample_size - 1) / population_size))

    return int(sample_size)

In [11]:
def get_max_n_chars(text, max_n):
  text_length = len(text)
  if text_length <= max_n:
    return text
  else:
    return text[:max_n]

def create_new_file(filename, dir_name, issue_type, summary, description, status, comments):
  try:
    filename = dir_name + '/' + filename

    if issue_type is None:
      issue_type = ''
    if summary is None:
      summary = ''
    if description is None:
      description = ''
    if status is None:
      status = ''
    if comments is None:
      comments = ''

    with open(filename, mode='w') as f_issue:
      f_issue.write(f'issue_type: {issue_type} \n')
      f_issue.write(f'summary: {summary} \n')
      f_issue.write(f'description: {get_max_n_chars(text=description, max_n=1000)} \n')
      f_issue.write(f'status: {status} \n')
      f_issue.write(f'comments: {get_max_n_chars(text=comments, max_n=4000)} \n')
    print(f'File {filename} created with success!')

  except Exception as ex:
    print(f'Erro ao criar arquivo: {str(ex)}')


In [53]:
# Seleciona randomicamente os issues para inspeção
def select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date):
  lista_issues_inspecao = []
  dict_issues_para_inspecao = {}
  list_issue_key = df_issues_in_commits_with_critical_classes.issue_key.to_list()
  list_issue_key = list(set(list_issue_key))
  sample_issues = random.choices(list_issue_key, k=sample_size)
  dict_issues_para_inspecao[my_date] = sample_issues
  print(f'{len(sample_issues)} para inspeção manual')

  date_file_name = my_date.split('/')
  date_file_name = date_file_name[0] + date_file_name[1] + date_file_name[2]
  file_name = 'issues_inspecao_' + date_file_name + '.txt'
  with open(file_name, mode='w') as f_temp:
    for v in dict_issues_para_inspecao[my_date]:
      elemento = v + ','
      f_temp.write(elemento)
  print(f'Relação de Issues salvos em {my_date} para inspeção.')
  return sample_issues

# Gera os arquivos .txt de cada issue selecionado para inspeção
def generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes):
  contador = 0
  my_dir_name = 'my_issues'
  if not os.path.exists(my_dir_name):
    os.makedirs(my_dir_name)

  total_of_issues = df_issues_in_commits_with_critical_classes.shape[0]
  for index in tqdm.tqdm(df_issues_in_commits_with_critical_classes.index, total=total_of_issues, desc='Analyzing issues'):
    for issue in sample_issues:
      if df_issues_in_commits_with_critical_classes.issue_key[index] == issue:
        create_new_file(filename=df_issues_in_commits_with_critical_classes.issue_key[index], dir_name=my_dir_name, issue_type=df_issues_in_commits_with_critical_classes.issue_type[index], summary=df_issues_in_commits_with_critical_classes.summary[index], description=df_issues_in_commits_with_critical_classes.description[index], status=df_issues_in_commits_with_critical_classes.status[index], comments=df_issues_in_commits_with_critical_classes.comments[index])
        contador += 1
  print(f'Foram criados {contador} arquivos para inspeção')

# 1. Extrai commits de um repositório de código

In [14]:
start_date = datetime(2009, 5, 19, 0, 0, 0)
end_date = datetime(2023, 10, 4, 0, 0, 0)
extract_all_commits = get_all_commits_by_range(initial_date=start_date, final_date=end_date, repository_name=my_repository)

all_commits = extract_all_commits[0]
total_all_commits = extract_all_commits[1]

print(f'Total de commits extraídos: {total_all_commits}')

Wait...


Progress commit anlysis: 100%|██████████| 26906/26906 [23:18<00:00, 19.24it/s]

Total de commits extraídos: 26906


## Filtra os commits com classes críticas

In [15]:
print(f'Analisa a faixa de commits entre: initial_date={str(start_date)}, final_date={str(end_date)}')
print('Registra apenas os commits que contem pelo menos um arquivo crítico')
filter_commits_with_critical_files = get_commits_by_range_and_critical_files(initial_date=start_date, final_date=end_date, critical_files=lista_arquivos_criticos, repository_name=my_repository)
commits_with_critical_files = filter_commits_with_critical_files[0]
total_of_commits_with_critical_files = filter_commits_with_critical_files[1]

print(f'Total de commits com classes críticas: {total_of_commits_with_critical_files}')

Analisa a faixa de commits entre: initial_date=2009-05-19 00:00:00, final_date=2023-10-04 00:00:00
Registra apenas os commits que contem pelo menos um arquivo crítico
Wait...


Progress critical files commit anlysis: 100%|██████████| 26906/26906 [13:04<00:00, 34.29it/s]

Total de commits com classes críticas: 26906


### Converte os commits com classes críticas para dataframe

In [16]:
df_commits_with_critical_files = convert_commits_to_dataframe(dict_of_commits=commits_with_critical_files)
df_commits_with_critical_files

,hash,msg,date,lines,files,critical_files,modified_files,diff_files,diff_files_modified_files
0,5128a9a453d64bfe1ed978cf9ffed27985eeef36,HADOOP-4687 Moving src directories on branch\n...,19/5/2009,67543,352,"[Configuration.java, FSDataOutputStream.java, ...","[core-default.xml, HadoopVersionAnnotation.jav...","[{'Configuration.java': '@@ -0,0 +1,1326 @@ +/...","[@@ -0,0 +1,444 @@\n+<?xml version=""1.0""?>\n+<..."
1,bcd64325a11cb0dd5096ffc093d0ffa68c4fcc58,"Merged src/core, src/test/core, src/contrib/ec...",15/6/2009,1799,37,[StringUtils.java],"[hadoop-ec2-init-remote.sh, launch-hadoop-slav...","[{'StringUtils.java': '@@ -88,7 +88,8 @@ publi...","[@@ -17,7 +17,9 @@ MASTER_HOST=%MASTER_HOST% #..."
2,b12d765467fd9a4447c473d613d92883fb09c76b,HADOOP-4687. Merge -r 784663:785643 from trunk...,17/6/2009,73,4,[StringUtils.java],"[CHANGES.txt, LocalDirAllocator.java, StringUt...","[{'StringUtils.java': '@@ -677,4 +677,24 @@ pu...","[@@ -68,6 +68,9 @@ Trunk (unreleased changes)\..."
3,a13237975d02b3db913b95845d4b8d6d22f2bac7,HADOOP-2366. Support trimmed strings in Config...,30/6/2009,98,4,"[Configuration.java, StringUtils.java]","[CHANGES.txt, Configuration.java, StringUtils....","[{'Configuration.java': '@@ -31,6 +31,7 @@ im...","[@@ -468,6 +468,9 @@ Trunk (unreleased changes..."
4,3200b2ec588dfa5c50f1ec6192ff93ab9187f82d,HADOOP-6161. Add get/setEnum methods to Config...,20/7/2009,43,3,[Configuration.java],"[CHANGES.txt, Configuration.java, TestConfigur...","[{'Configuration.java': '@@ -595,6 +595,30 @@ ...","[@@ -480,6 +480,8 @@ Trunk (unreleased changes..."
...,...,...,...,...,...,...,...,...,...
2771,d5334fa76170b99f1ddd6b307482d226da12f1a9,YARN-6537. Running RM tests against the Router...,3/9/2023,816,12,[NodeManager.java],"[YarnConfiguration.java, NodeManager.java, pom...","[{'NodeManager.java': '@@ -1074,4 +1074,9 @@ p...","[@@ -517,7 +517,7 @@ public static boolean isA..."
2772,4652d22b9195d35eac4d7e02d1f99ebc6a5835c7,HDFS-17178: BootstrapStandby needs to handle R...,12/9/2023,256,11,[FSNamesystem.java],"[pom.xml, PBHelper.java, HdfsServerConstants.j...","[{'FSNamesystem.java': '@@ -1898,9 +1898,7 @@ ...","[@@ -219,6 +219,10 @@ https://maven.apache.org..."
2773,ecee022e49269bd6612d32695c6b46a95c6bf11e,HDFS-17197. Show file replication when listing...,25/9/2023,25,1,[FSNamesystem.java],[FSNamesystem.java],"[{'FSNamesystem.java': '@@ -6131,15 +6131,20 @...","[@@ -6131,15 +6131,20 @@ void releaseBackupNod..."
2774,26a5f38250de1ac4978427d74c200a968f9a0b65,HDFS-17204. EC: Reduce unnecessary log when pr...,26/9/2023,6,1,[BlockManager.java],[BlockManager.java],"[{'BlockManager.java': '@@ -4201,6 +4201,12 @@...","[@@ -4201,6 +4201,12 @@ private void chooseExc..."


In [17]:
df_commits_with_critical_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2776 entries, 0 to 2775
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   hash                       2776 non-null   object
 1   msg                        2776 non-null   object
 2   date                       2776 non-null   object
 3   lines                      2776 non-null   int64 
 4   files                      2776 non-null   int64 
 5   critical_files             2776 non-null   object
 6   modified_files             2776 non-null   object
 7   diff_files                 2776 non-null   object
 8   diff_files_modified_files  2776 non-null   object
dtypes: int64(2), object(7)
memory usage: 195.3+ KB


In [40]:
df_commits_with_critical_files.to_excel('hadoop_commits_with_critical_classes.xlsx', index=False)

# 2. Extrai issues de um issue tracker

## Configura ambiente para acessar Issue Tracker

In [24]:
# Initialize the Jira connection
print('Initialize the Jira connection')
jira = JIRA(JIRA_SERVER, basic_auth=(username, password))

# Create a JiraUtils instance
print('Create a JiraUtils instance')
jira_utils = JiraUtils(ISSUE_TRACKER_PROJECT, jira)

date1 = (2009, 5, 19)
date2 = (2023, 10, 4)
distance = 120
print('Define date intervals')
print(f'From: {str(date1)} to: {date2}, by: {distance} days of distance.')

Initialize the Jira connection
Create a JiraUtils instance
Define date intervals
From: (2009, 5, 19) to: (2023, 10, 4), by: 120 days of distance.


## Recupera os issues do período dado

In [25]:
# Fetch issues using date intervals
print('Fetch issues using date intervals')
block_of_issues = jira_utils.get_list_of_block_issues_by_dates(date1, date2, distance)

Fetch issues using date intervals
Aguarde...


Progress Message Analysis:   2%|▏         | 1/44 [00:03<02:38,  3.69s/it]

Range: project=HADOOP and created>="2009/05/19" and created<="2009/09/15", qtd issues: 244


Progress Message Analysis:   5%|▍         | 2/44 [00:07<02:41,  3.85s/it]

Range: project=HADOOP and created>="2009/09/16" and created<="2010/01/13", qtd issues: 211


...

Range: project=HADOOP and created>="2022/11/07" and created<="2023/03/06", qtd issues: 118


Progress Message Analysis:  98%|█████████▊| 43/44 [03:26<00:02,  2.80s/it]

Range: project=HADOOP and created>="2023/03/07" and created<="2023/07/04", qtd issues: 134


Progress Message Analysis: 100%|██████████| 44/44 [03:28<00:00,  4.75s/it]

Range: project=HADOOP and created>="2023/07/05" and created<="2023/11/01", qtd issues: 161
2023-11-09 13:31:26.415187
Tempo da consulta: 0:03:28.809376


In [26]:
# Concatenate the block of issues into a single list
print('Concatenate the block of issues into a single list')
all_issues = jira_utils.concatenate_block_of_issues(block_of_issues)

Concatenate the block of issues into a single list
Total de issues recuperados: 12247


In [27]:
all_real_issues = analyze_jira_all_issues(ISSUE_TRACKER_PROJECT, all_issues)

Progress jira all issues analysis: 100%|██████████| 12247/12247 [00:00<00:00, 65052.06it/s]


### Converte os issues recuperados para um dataframe

In [28]:
df_all_reall_issues_in_commits_detailed = convert_issues_to_dataframe(all_real_issues)
df_all_reall_issues_in_commits_detailed

,issue_key,issue_type,status,priority,summary,description,comments,created_date,resolved_date
0,HADOOP-12459,Improvement,Patch Available,Minor,Implement moveToLocal HDFS command,Surprisingly executing HDFS FsShell command -...,{code}\n$ bin/hadoop fs -moveToLocal\nmoveToLo...,2009-07-11,NaT
1,HADOOP-6272,Improvement,Resolved,Major,Incorrect UserName at Solaris because it has n...,"Solaris enviroment has no __whoami__ command, ...",Is it possible for you to find out the name of...,2009-07-27,2014-07-23
2,HADOOP-6255,New Feature,Closed,Major,Create an rpm integration project,We should be able to create RPMs for Hadoop re...,"Hi,\n\nI would suggest the other way around: c...",2009-09-11,2011-05-27
3,HADOOP-6254,Bug,Closed,Major,s3n fails with SocketTimeoutException,If a user's map function is CPU intensive and ...,Here is my proposed fix. This patch doesn't in...,2009-09-11,2009-12-09
4,HADOOP-6253,New Feature,Resolved,Minor,Add a Ceph FileSystem interface.,The experimental distributed filesystem Ceph d...,I've attached a patch which includes the CephF...,2009-09-11,2014-11-04
...,...,...,...,...,...,...,...,...,...
12242,HADOOP-18795,Sub-task,Resolved,Minor,s3a DelegationToken plugin to expand return ty...,Change the binding result of s3a DT services b...,"steveloughran opened a new pull request, #5821...",2023-07-08,2023-07-20
12243,HADOOP-18794,Improvement,Resolved,Major,ipc.server.handler.queue.size missing from cor...,This config should be documented. It is necess...,"YuanbenWang opened a new pull request, #5819:\...",2023-07-08,2023-07-11
12244,HADOOP-18793,Bug,Resolved,Minor,S3A StagingCommitter does not clean up staging...,When setting up StagingCommitter and its inter...,I'm guessing ${UUID} directory is preserved on...,2023-07-06,2023-07-08
12245,HADOOP-18792,Sub-task,In Progress,Major,s3a prefetching to use split start/end options...,the bundled hadoop record readers pass the spl...,,2023-07-05,NaT


In [31]:
colunas = ['issue_key',	'issue_type', 'status', 'priority', 'summary']
#df_all_reall_issues_in_commits_detailed[colunas].to_excel('hadoop_all_issues_in_commits.xlsx', index=False)

In [32]:
df_all_reall_issues_in_commits_detailed['time_resolution'] = df_all_reall_issues_in_commits_detailed['resolved_date'] - df_all_reall_issues_in_commits_detailed['created_date']

In [41]:
print('Gera arquivo .csv hadoop_all_issues_in_commits')
colunas = ['issue_key', 'issue_type', 'status', 'summary','created_date', 'resolved_date', 'time_resolution']
df_all_reall_issues_in_commits_detailed[colunas].to_csv('hadoop_all_issues_in_commits.csv', index=False)

Gera arquivo .csv hadoop_all_issues_in_commits


### Recupera Issues com referência nos commits com classes críticas

In [34]:
dict_issues_in_commits, df_issues_in_commits_with_critical_classes = get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files)
df_issues_in_commits_with_critical_classes

,issue_key,issue_type,status,priority,summary,description,comments,created_date,resolved_date,time_resolution
5,HADOOP-6252,Improvement,Closed,Major,Provide method to determine if a deprecated ke...,HADOOP-6105 provided a method to deprecate con...,Patch:\n * Implements new method to see if d...,2009-09-10,2009-09-11,1 days
13,HADOOP-6243,Bug,Closed,Blocker,NPE in handling deprecated configuration keys.,I run TestFileCreation in Eclipse and get a Nu...,Running the same test using ant command line w...,2009-09-07,2009-09-08,1 days
21,HADOOP-6233,Improvement,Closed,Major,Changes in common to rename the config keys as...,This jira tracks the code changes required in ...,The attached patch is created after merging th...,2009-09-02,2009-09-19,17 days
26,HADOOP-6227,Bug,Closed,Major,Configuration does not lock parameters marked ...,"A use-case: Recently, we stumbled on a bug tha...",The code in configuration looks like this:\n\n...,2009-09-01,2009-09-01,0 days
63,HADOOP-6184,Bug,Closed,Major,Provide a configuration dump in json format.,Configuration dump in json format.,In order to generate the dump in standard for...,2009-08-10,2009-08-24,14 days
...,...,...,...,...,...,...,...,...,...,...
11628,HADOOP-18177,Sub-task,Resolved,Major,document use and architecture design of prefet...,Document S3PrefetchingInputStream for users (...,[~stevel@apache.org] have added an architectur...,2022-03-25,2022-04-26,32 days
11630,HADOOP-18175,Sub-task,Resolved,Major,test failures with prefetching s3a input stream,identify and fix all test regressions from the...,h3. ITestS3AContractUnbuffer\r\n\r\nfix: getPo...,2022-03-25,2022-05-05,41 days
11698,HADOOP-18469,Improvement,Resolved,Major,Add XMLUtils methods to centralise code that c...,Relates to HDFS-16766\r\n\r\nThere are other p...,"pjfanning opened a new pull request, #4940:\nU...",2022-09-27,2022-10-07,10 days
11786,HADOOP-18379,Sub-task,Resolved,Major,rebase feature/HADOOP-18028-s3a-prefetch to trunk,"rebase to trunk, fix conflicts and tests, forc...",,2022-07-28,2022-08-03,6 days


In [35]:
df_issues_in_commits_with_critical_classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 5 to 11855
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   issue_key        239 non-null    object         
 1   issue_type       239 non-null    object         
 2   status           239 non-null    object         
 3   priority         239 non-null    object         
 4   summary          239 non-null    object         
 5   description      239 non-null    object         
 6   comments         239 non-null    object         
 7   created_date     239 non-null    datetime64[ns] 
 8   resolved_date    236 non-null    datetime64[ns] 
 9   time_resolution  236 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), object(7), timedelta64[ns](1)
memory usage: 20.5+ KB


In [36]:
df_issues_in_commits_with_critical_classes['time_resolution'] = df_issues_in_commits_with_critical_classes['resolved_date'] - df_issues_in_commits_with_critical_classes['created_date']

In [42]:
df_issues_in_commits_with_critical_classes[colunas].to_excel('hadoop_issues_in_commits_with_critical_classes.xlsx', index=False)

In [43]:
print('Gera arquivo .csv kafka_issues_in_commits_arquivos_criticos')
df_issues_in_commits_with_critical_classes[colunas].to_csv('hadoop_issues_in_commits_with_critical_classes.csv', index=False)

Gera arquivo .csv kafka_issues_in_commits_arquivos_criticos


# Seleciona amostra dos Issues

In [60]:
confidence_level = 0.98
margin_of_error = 0.05
population_proportion = 0.8
population_size = len(df_issues_in_commits_with_critical_classes)

sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
print(f'População de issues: {population_size}')
print(f"Sample size para inspeção: {sample_size}")

População de issues: 239
Sample size para inspeção: 141


In [54]:
sample_issues = select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='09/11/2023')

239 para inspeção manual
Relação de Issues salvos em 09/11/2023 para inspeção.


In [55]:
generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes)

Analyzing issues:  27%|██▋       | 65/239 [00:00<00:00, 318.97it/s]

File my_issues/HADOOP-6252 created with success!
File my_issues/HADOOP-6252 created with success!
File my_issues/HADOOP-6184 created with success!
File my_issues/HADOOP-6184 created with success!
File my_issues/HADOOP-6165 created with success!
File my_issues/HADOOP-6165 created with success!
File my_issues/HADOOP-6161 created with success!
File my_issues/HADOOP-6161 created with success!
File my_issues/HADOOP-6161 created with success!
File my_issues/HADOOP-6103 created with success!
File my_issues/HADOOP-6103 created with success!
File my_issues/HADOOP-6471 created with success!
File my_issues/HADOOP-6443 created with success!
File my_issues/HADOOP-6443 created with success!
File my_issues/HADOOP-6420 created with success!
File my_issues/HADOOP-6346 created with success!
File my_issues/HADOOP-6323 created with success!
File my_issues/HADOOP-6323 created with success!
File my_issues/HADOOP-6323 created with success!
File my_issues/HADOOP-6313 created with success!
File my_issues/HADOO

Analyzing issues:  55%|█████▍    | 131/239 [00:00<00:00, 322.51it/s]

File my_issues/HADOOP-7993 created with success!
File my_issues/HADOOP-7993 created with success!
File my_issues/HADOOP-8632 created with success!
File my_issues/HADOOP-8608 created with success!
File my_issues/HADOOP-8573 created with success!
File my_issues/HADOOP-8525 created with success!
File my_issues/HADOOP-8525 created with success!
File my_issues/HADOOP-8524 created with success!
File my_issues/HADOOP-8524 created with success!
File my_issues/HADOOP-8524 created with success!
File my_issues/HADOOP-8524 created with success!
File my_issues/HADOOP-8524 created with success!
File my_issues/HADOOP-8362 created with success!
File my_issues/HADOOP-8362 created with success!
File my_issues/HADOOP-8359 created with success!
File my_issues/HADOOP-8349 created with success!
File my_issues/HADOOP-8952 created with success!
File my_issues/HADOOP-8821 created with success!
File my_issues/HADOOP-8780 created with success!
File my_issues/HADOOP-8780 created with success!
File my_issues/HADOO

Analyzing issues:  82%|████████▏ | 197/239 [00:00<00:00, 314.04it/s]

File my_issues/HADOOP-12317 created with success!
File my_issues/HADOOP-12317 created with success!
File my_issues/HADOOP-12317 created with success!
File my_issues/HADOOP-12097 created with success!
File my_issues/HADOOP-12097 created with success!
File my_issues/HADOOP-12097 created with success!
File my_issues/HADOOP-11901 created with success!
File my_issues/HADOOP-11901 created with success!
File my_issues/HADOOP-11852 created with success!
File my_issues/HADOOP-12457 created with success!
File my_issues/HADOOP-12457 created with success!
File my_issues/HADOOP-12457 created with success!
File my_issues/HADOOP-12457 created with success!
File my_issues/HADOOP-12437 created with success!
File my_issues/HADOOP-13770 created with success!
File my_issues/HADOOP-12916 created with success!
File my_issues/HADOOP-12862 created with success!
File my_issues/HADOOP-12862 created with success!
File my_issues/HADOOP-12639 created with success!
File my_issues/HADOOP-13442 created with success!


Analyzing issues: 100%|██████████| 239/239 [00:00<00:00, 318.91it/s]

File my_issues/HADOOP-16906 created with success!
File my_issues/HADOOP-16830 created with success!
File my_issues/HADOOP-17123 created with success!
File my_issues/HADOOP-17088 created with success!
File my_issues/HADOOP-17079 created with success!
File my_issues/HADOOP-16951 created with success!
File my_issues/HADOOP-16951 created with success!
File my_issues/HADOOP-16951 created with success!
File my_issues/HADOOP-17358 created with success!
File my_issues/HADOOP-17358 created with success!
File my_issues/HADOOP-17288 created with success!
File my_issues/HADOOP-17424 created with success!
File my_issues/HADOOP-17613 created with success!
File my_issues/HADOOP-17963 created with success!
File my_issues/HADOOP-17963 created with success!
File my_issues/HADOOP-17959 created with success!
File my_issues/HADOOP-17959 created with success!
File my_issues/HADOOP-17957 created with success!
File my_issues/HADOOP-18028 created with success!
File my_issues/HADOOP-18028 created with success!


In [56]:
!zip -r my_issues.zip my_issues

  adding: my_issues/ (stored 0%)
  adding: my_issues/HADOOP-10402 (deflated 73%)
  adding: my_issues/HADOOP-6471 (deflated 67%)
  adding: my_issues/HADOOP-6252 (deflated 62%)
  adding: my_issues/HADOOP-10682 (deflated 67%)
  adding: my_issues/HADOOP-6323 (deflated 59%)
  adding: my_issues/HADOOP-7524 (deflated 57%)
  adding: my_issues/HADOOP-9582 (deflated 67%)
  adding: my_issues/HADOOP-16596 (deflated 70%)
  adding: my_issues/HADOOP-9688 (deflated 62%)
  adding: my_issues/HADOOP-13706 (deflated 78%)
  adding: my_issues/HADOOP-17088 (deflated 55%)
  adding: my_issues/HADOOP-6165 (deflated 60%)
  adding: my_issues/HADOOP-6269 (deflated 62%)
  adding: my_issues/HADOOP-10482 (deflated 68%)
  adding: my_issues/HADOOP-8359 (deflated 75%)
  adding: my_issues/HADOOP-12916 (deflated 75%)
  adding: my_issues/HADOOP-18229 (deflated 76%)
  adding: my_issues/HADOOP-10208 (deflated 73%)
  adding: my_issues/HADOOP-6184 (deflated 57%)
  adding: my_issues/HADOOP-6312 (deflated 74%)
  adding: my_issue